## Single track model

Searching for a single track model, I found this wikipedia page (which appears to be only in german !?) (https://de.wikipedia.org/wiki/Einspurmodell)

This model assumes a fixed speed v. The vehicle rotates with speed psi_dot. The velocity v is not aligned to the vehicle but has an angle beta towards the main axis.
See below for an image.

First, I've set up a simulation of the dynamics of this system in sympy.
Then, as a second step, I'm deriving a very simple single track model from scratch. Please note: This second model is probably not the canonical "single track model". But I think it would be more comfortable to use.
So my goal is to be able to compare the behaviours.

TODO: so far I'm using a runge-kutta method to step through time. That is not a good idea. Switch to this: https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models


### Part 1, proper single track model

In [1]:
import numpy as np
from sympy import *

In [2]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/2/24/EinspurKinematik.png")

In [3]:
# parameters of the model
m, theta, l_v, l_h, c_v, c_h, i_S = symbols("m, theta, l_v, l_h, c_v, c_h, i_S")

# some sensible reference values, taken from the wikipedia article
parameters = {
m : 1550,    # kg
theta : 2800, # kg m^2
l_v : 1.344,  # m
l_h : 1.456,  # m
c_v : 75000,  # N / rad
c_h : 150000, # N / rad
i_S : 16
}

In [4]:
# in the single track model, v is assumed to be constant
# here, I'm keeping v symbolic, so that it's possible to plug in different values of v
v = symbols("v")

# state of the model is  x = [beta, psi_dot]
# input of the model is u = [delta / i_S]
beta, psi_dot = symbols("beta psi_dot")
delta = symbols("delta")

In [5]:
# x_dot = A * x + B * u
# here are the corresponding matrices A and B
A = Matrix([[-(c_v + c_h)/(m*v), (m*v**2 - (c_h*l_h - c_v*l_v))/(m*v**2)],
           [- (c_h*l_h - c_v*l_v)/theta, -(c_h*l_h**2 + c_v*l_v**2)/(theta*v)]])

B = Matrix([[-c_v / (m*v)], [c_v*l_v/theta]])

In [6]:
# state space representation: dx = A*x + B*u
def dstate(state, u):
    return A @ state + B@u

In [7]:
# a runge kutta implementation
T = symbols("T")

state = Matrix([beta, psi_dot])
u = Matrix([delta / i_S])

# runge kutta
k1 = dstate(state, u)
k2 = dstate(state + T/2*k1, u)
k3 = dstate(state + T/2*k2, u)
k4 = dstate(state + T*k3, u)

rk = state + T/6 * (k1 + 2*k2 + 2*k3 + k4)

In [8]:
# a sample evaluation of the runge kutta setup

# here we define values to be used for states and inputs
# they will be substituted into the symbolic expression
beta_val = 0
psi_dot_val = 0
delta_val = 0.05
v_val = 10
dT = 1

substitutions = {
    beta: beta_val, 
    psi_dot : psi_dot_val, 
    delta: delta_val, 
    v:v_val,
    T: dT
}

# we also need to substitute the parameters from above
rk.subs({**substitutions, **parameters}).evalf()

Matrix([
[1.89157784341162],
[1.78230636680725]])

### part 2, simplified version

In [9]:
# time
t = symbols("t")

# parameters are mass and length
# also a parameter is the steering delay
m, l, T_delta = symbols("m l T_delta")

# state is velocity at the back-wheel
# and the current steering angle
v, delta= symbols("v delta")

# input is target steering angle and force for acceleration
delta_target, F = symbols("delta_target F")

In [10]:
v = v(t)

In [11]:
# angular velocity can be derived from radius of curvature and velocity
# omega * R = v, tan(delta) = l/R
# => 1/R = tan(delta) / l
# => omega = v / R
omega = tan(delta) / l * v

In [12]:
# speed at the center of mass, in the middle of the vehicle, can be derived from speed at back wheel
# and angular velocity
v_mass = sqrt(v**2 + (omega * l/2)**2)

In [13]:
# the velocity of the center of mass can be derived by time
# this is the accleration
diff(v_mass, t)

(v(t)*tan(delta)**2*Derivative(v(t), t)/4 + v(t)*Derivative(v(t), t))/sqrt(v(t)**2*tan(delta)**2/4 + v(t)**2)

In [14]:
# but the above is not usable for us
# we need to set this equal to the acceleration and solve for the derivative of v
# the result is stored in the new symbol d_v
a = F/m
d_v = solve(diff(v_mass, t) -a, diff(v, t))
d_v

[2*F*sqrt((tan(delta)**2 + 4)*v(t)**2)/(m*(tan(delta)**2 + 4)*v(t))]

In [15]:
# as you can see above, there is a list of solutions
# it contains just one element, though
# we are only interested in this single solution
d_v = d_v[0]

In [16]:
# for the dynamics of the steering angle,
# we model a simple PT behaviour
d_delta = (delta_target - delta) / T_delta
d_delta

(-delta + delta_target)/T_delta

In [17]:
# linearization in state space:
state = Matrix([v, delta])
d_state = Matrix([d_v, d_delta])
u = Matrix([F, delta_target])

A = d_state.jacobian(state)
B = d_state.jacobian(u)

In [18]:
A

Matrix([
[0, -F*sqrt((tan(delta)**2 + 4)*v(t)**2)*(2*tan(delta)**2 + 2)*tan(delta)/(m*(tan(delta)**2 + 4)**2*v(t))],
[0,                                                                                            -1/T_delta]])

In [19]:
B

Matrix([
[2*sqrt((tan(delta)**2 + 4)*v(t)**2)/(m*(tan(delta)**2 + 4)*v(t)),         0],
[                                                               0, 1/T_delta]])